In [24]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
import uvicorn
import pickle
import re
import spacy



from fuzzywuzzy import fuzz, process
from rapidfuzz import process as rprocess, fuzz as rfuzz
import pandas as pd

projects_list_final = ['Broadway Heights','Prime City','Icon Valley','Zameen OPAL','Defence Raya Fairways','Zee Avenue','Zameen Ace Mall','Palm Dreams',
'The Opus- Luxury Residencies','Bali Business Boulevard','Zameen Ace Homes','City Star Residencia','City Star Shopping Centre','Mall 35','Canal Valley Daska',
'Zameen Aurum','Pace Circle','Roman Grove','The Mega Mall & Residency','3 Jays Tower','River Courtyard','Zameen Quad','Gulberg City Center','RJS-Lifestyle Residencia',
'Sitara Icon','Residence 15','Sitara Serene','Mall of Gujrat','Grand Gallery','DB32','Amanah Noor Residence','Grande Palladium','The Edge Mall',
'J Heights','Tomorrowland','Highland Villas','Pearl One Residencies','Downtown Rumanza','De Orion Mall','Golf View Rumanza','Serenity Heights',
'Union Tower','Hyde Park One','Rockland Villas','Box Park II','Al Rafi Heights','Platinum Homes DHAM','102 by Icon-Residential Towers','ONE VH','Amanah Mall Service Apartments',
'201 Apartment','North Vista-II','Jinnah Square apartments','Aziz Excellency','River Hills 5','Swiss 99 Tower','River Courtyard Tower 2','Zameen NEO',
'Central Park Townhouses','Saeeda Residency','Nova City','Zen Apartments','Century Venture 1','Beach Resort','Expo Gold','Mountain Village Naran',
'RJs Square','Madison Square','Grand Orchard','Al Rauf Smart City','Rachna Pearl Hotel','Al-Mannan Spanish Villas','Platinum Villas DHAM','Zulekha Residency',
'Boulevard Heights','R&M Tower','Barki Orchard','Smart City Housing Scheme','Spanish Homes','Amber Prime','Damaan City','Bin Qasim Trade Centre','Q High Street','Dream Homes',
'Zameen Phoenix','Zameen Jade','Premier One','Palm Green Villas','Spanish Villas','360 Residences',
'Garden City','Prime City 2','Green Scape Farms','Airport Enclave', 'Spanish', 'Damaan', '360 Res', 'Premium One', 'The Man City', 'Smart City', 'Premier 1', 'Premium 1']
stop_projects = ['residence', 'apartment', 'residencies', 'one', 'zameen', 'valley']

projects_list_final = [item.lower() for item in projects_list_final]
def contains(list, value):
    counter = 0
    for i in list:
        if i.__contains__(value):
            counter +=1
        return counter
class ProjectNameExtraction:
    def __init__(self, projects_names_list, common_projects_words):
        self.projects_list = projects_names_list
        self.stop_words = common_projects_words
        self.results = None
        self.text_input = None
    def project_name(self, text: str) -> str:
        if type(text) == float: #handling errors for when transcription is invalid
            return None
        text = text.replace("\\'", "")
        text = text.replace("'", "")
        text = re.sub(r'[\n.]', '', text)
        text = text.lower()
        threshold = 85
        results = []
        projects_extracted = []
        for name in self.projects_list:
            matches = process.extractOne(name, [text], scorer = rfuzz.partial_ratio)
            if any(common_word in text for common_word in self.stop_words):
                threshold = 87
            if name == 'zen apartments':
                threshold = 95
            elif name == 'union tower':
                threshold = 90
            elif name == 'prime city 2':
                threshold = 95
            elif name == 'j heights':
                threshold = 95        
            elif name == 'tomorrowland':
                threshold = 98
            if matches[1] > threshold:  # Set your threshold for a match
            #matches[1] is the identified match, search string is the project name, i is the row number
                results.append([name, matches[1]])
                projects_extracted.append(name)
        premium_one_list = ['premium one', 'premier 1', 'premium 1', 'premier one']
        damaan_list = ['damaan', 'damaan city', 'the man city']
        spanish_list = ['spanish villas', 'spanish homes', 'spanish']   
        prem_counter = 0 
        for value in premium_one_list:
            if value in projects_extracted: prem_counter +=1
        if prem_counter >1: 
            projects_extracted = [item for item in projects_extracted if item not in premium_one_list]
            projects_extracted.append('Premier One')
        damaan_counter = 0 
        for value in damaan_list:
            if value in projects_extracted: damaan_counter +=1
        if damaan_counter >1: 
            projects_extracted = [item for item in projects_extracted if item not in damaan_list]
            projects_extracted.append('damaan city')
        spanish_counter = 0 
        for value in spanish_list:
            if value in projects_extracted: prem_counter +=1
        if prem_counter ==3: 
            projects_extracted = [item for item in projects_extracted if item not in spanish_list]
            projects_extracted.append('spanish villas')
            projects_extracted.append('spanish homes')
        elif spanish_counter == 2 and 'spanish' in projects_extracted:
            projects_extracted.remove('spanish')
        # if '360 Res' in projects_extracted: projects_extracted.remove('360 Res') garden city, airport enclave, zameen quad, smart city
        self.results = projects_extracted
        self.text_input = text
        return projects_extracted
    def spacy_backcheck(self):
        nlp = spacy.load('model-finetuned/model-finetuned')
        doc = nlp(self.text_input)
        entities = [{"text": ent.text, "label": ent.label_} for ent in doc.ents]
        for entity in entities:
            matches = process.extractOne(entity['text'], projects_list_final, scorer = rfuzz.partial_ratio)
            if int(matches[1]) > 80 and matches[0] in self.projects_list and matches[0] not in self.results:
                self.results.append(matches[0])
                print(f'{matches[0]} was not extracted by the fuzzy wuzzy function but is in the transcription')
        print(self.results)


In [26]:
instant = ProjectNameExtraction(projects_list_final, stop_projects)
instant.project_name('yes my project is spanish velazques I live in donton rumanza I love prime city. We have many in azaz excellency. river cortyard is available')
print(instant.results)
instant.spacy_backcheck()

['prime city', 'river courtyard', 'aziz excellency', 'spanish']
downtown rumanza was not extracted by the fuzzy wuzzy function but is in the transcription
['prime city', 'river courtyard', 'aziz excellency', 'spanish', 'downtown rumanza']
